In [1]:
pip install mesa


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.8 MB 7.3 MB/s eta 0:00:01
     -------------- ------------------------- 0.7/1.8 MB 8.4 MB/s eta 0:00:01
     -------------------------------- ------- 1.5/1.8 MB 13.6 MB/s eta 0:00:01
     ---------------------------------------  1.8/1.8 MB 11.7 MB/s eta 0:00:01
     ---------------------------------------- 1.8/1.8 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
     - -------------------------------------- 0.6/13.1 MB 17.5 MB/s eta 0:00:01
     -- ------------------------------------- 1.0/13.1 MB 15.4 MB/s eta 0:00:01
     ---- ----------------------------------- 1.4/13.1 MB 15.1 MB/s eta 0:00:01
     ----- ---------------------------------- 1.9/13.1 MB 13.3 MB/s eta 0:00:01
     ------- -------------------------------- 2.4/13.1 MB 14.2 MB/s eta 0:00:01
     --------- ------------------------------ 3.2/13.1 MB 14.6 

In [4]:
import random
import statistics
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, age, shopping_propensity):
        super().__init__(unique_id, model)
        self.age = age
        self.shopping_propensity = shopping_propensity

    def step(self):
        # Update agent's age
        self.age += 1

        # Consumer agents take action in each step (e.g., shop for goods)
        if self.age >= 65:
            # Retirees are more picky shoppers
            if random.random() < self.shopping_propensity * 0.8:
                # Retiree makes a purchase decision
                pass
        else:
            # Unemployed workers are more picky shoppers compared to the employed workers
            if random.random() < (self.shopping_propensity * 0.8) \
            if isinstance(self, LaborAgent) and not self.employed else self.shopping_propensity:
                # Labor force agents make a purchase decision
                pass
i = 1            
age = random.randint(65, 100)
propensity = random.uniform(0.1, 1.0)
consumer = ConsumerAgent(i, self, age, propensity)

NameError: name 'self' is not defined

In [2]:
import random
import statistics
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, age, shopping_propensity):
        super().__init__(unique_id, model)
        self.age = age
        self.shopping_propensity = shopping_propensity

    def step(self):
        # Update agent's age
        self.age += 1

        # Consumer agents take action in each step (e.g., shop for goods)
        if self.age >= 65:
            # Retirees are more picky shoppers
            if random.random() < self.shopping_propensity * 0.8:
                # Retiree makes a purchase decision
                pass
        else:
            # Unemployed workers are more picky shoppers compared to the employed workers
            if random.random() < (self.shopping_propensity * 0.8) \
            if isinstance(self, LaborAgent) and not self.employed else self.shopping_propensity:
                # Labor force agents make a purchase decision
                pass

class FirmAgent(Agent):
    def __init__(self, unique_id, model, price):
        super().__init__(unique_id, model)
        self.price = price

    def step(self):
        # Firm agents take action in each step (e.g., produce and sell goods)
        unemployed_workers = [agent for agent in self.model.schedule.agents \
                              if isinstance(agent, LaborAgent) and not agent.employed]

        if len(unemployed_workers) > 0:
            # Hire one unemployed worker (randomly)
            worker = random.choice(unemployed_workers)
            worker.employed = True
            # Produce goods using the worker and other resources
            pass
        else:
            # No vacant job, the firm cannot produce more goods
            pass

        # Set the price for the goods
        self.price = random.uniform(10, 50)

class LaborAgent(Agent):
    def __init__(self, unique_id, model, age):
        super().__init__(unique_id, model)
        self.age = age
        self.employed = False if age == 16 else True

    def step(self):
        # Update agent's age
        self.age += 1

        # Labor agents take action in each step (e.g., job search, getting fired)
        if self.age >= 65:
            # Retire the agent if they reach the retirement age
            self.model.schedule.remove(self)
        elif self.employed:
            # Agent has a job
            if random.random() < self.model.job_loss_probability:
                # Agent loses the job due to an exterior shock (e.g., economic crisis or technology advance)
                self.employed = False
        else:
            # Agent is unemployed and searches for a job
            if random.random() < self.model.job_search_probability:
                # Agent finds a new job
                self.employed = True

class EconomicModel(Model):
    def __init__(self, num_consumers, num_firms, num_labor_agents, \
                 job_search_probability, job_loss_probability, birth_rate, death_rate):
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(1, 1, False)
        self.datacollector = DataCollector(
            model_reporters={"UnemploymentRate": "get_unemployment_rate", "OldAgedDependencyRatio": "get_old_aged_dependency_ratio"}
        )
        self.job_search_probability = job_search_probability
        self.job_loss_probability = job_loss_probability
        self.birth_rate = birth_rate
        self.death_rate = death_rate

        # Create consumer agents (retirees)
        for i in range(num_consumers):
            age = random.randint(65, 100)
            propensity = random.uniform(0.1, 1.0)
            consumer = ConsumerAgent(i, self, age, propensity)
            self.schedule.add(consumer)
            self.grid.place_agent(consumer, (0, 0))

        # Create labor agents
        for i in range(num_labor_agents):
            age = random.randint(16, 64)
            propensity = random.uniform(0.1, 1.0)
            consumer = ConsumerAgent(i + num_consumers, self, age, propensity)
            self.schedule.add(consumer)
            self.grid.place_agent(consumer, (0, 0))

        # Create firm agents
        for i in range(num_firms):
            firm = FirmAgent(i, self, price=random.uniform(10, 50))
            self.schedule.add(firm)
            self.grid.place_agent(firm, (0, 0))

    def step(self):
        # Perform yearly population dynamics (birth and death)
        self.perform_population_dynamics()

        # Run agent step functions
        self.schedule.step()
        self.datacollector.collect(self)

    def perform_population_dynamics(self):
        # Perform birth and death rate calculations
        num_births = int(self.birth_rate * len([a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 16]))
        num_deaths = int(self.death_rate * len([a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 65]))
        
        # Create new labor agents (16-year-olds)
        for _ in range(num_births):
            new_agent = LaborAgent(self.next_id(), self, age=16)
            self.schedule.add(new_agent)
            self.grid.place_agent(new_agent, (0, 0))
        
        # Remove retired agents (aged 65 and over)
        retire_agents = [a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 65]
        for agent in retire_agents[:num_deaths]:
            self.schedule.remove(agent)

    def get_unemployment_rate(self):
        employed_agents = [agent for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.employed]
        total_labor_force = sum(1 for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.age >= 16 and agent.age <= 64)
        return 1 - len(employed_agents) / total_labor_force if total_labor_force > 0 else 0

    def get_old_aged_dependency_ratio(self):
        retirees = sum(1 for agent in self.schedule.agents if isinstance(agent, ConsumerAgent) and agent.age >= 65)
        labor_force = sum(1 for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.age >= 16 and agent.age <= 64)
        return labor_force / retirees if retirees > 0 else 0

# Set the number of agents for each type
num_consumers = 50
num_firms = 10
num_labor_agents = 100
job_search_probability = 0.1  # Probability of finding a job when unemployed
job_loss_probability = 0.05   # Probability of losing a job when employed
birth_rate = 0.02            # Birth rate (percentage of new agents each year)
death_rate = 0.01            # Death rate (percentage of agents in the retired group that die each year)

# Initialize the model
model = EconomicModel(num_consumers, num_firms, num_labor_agents, job_search_probability, job_loss_probability, birth_rate, death_rate)

# Run the model for a certain number of steps (e.g., 50 years)
for _ in range(50):
    model.step()

# Access data collected during the simulation
data = model.datacollector.get_model_vars_dataframe()
print(data)


Exception: Agent with unique id 0 already added to scheduler

In [5]:
import random
import statistics
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

class ConsumerAgent(Agent):
    def __init__(self, unique_id, model, age, shopping_propensity):
        super().__init__(unique_id, model)
        self.age = age
        self.shopping_propensity = shopping_propensity

    def step(self):
        # Update agent's age
        self.age += 1

        # Consumer agents take action in each step (e.g., shop for goods)
        if self.age >= 65:
            # Retirees are more picky shoppers
            if random.random() < self.shopping_propensity * 0.8:
                # Retiree makes a purchase decision
                pass
        else:
            # Unemployed workers are more picky shoppers compared to the employed workers
            if random.random() < (self.shopping_propensity * 0.8) if isinstance(self, LaborAgent) and not self.employed else self.shopping_propensity:
                # Labor force agents make a purchase decision
                pass

class FirmAgent(Agent):
    def __init__(self, unique_id, model, price):
        super().__init__(unique_id, model)
        self.price = price

    def step(self):
        # Firm agents take action in each step (e.g., produce and sell goods)
        unemployed_workers = [agent for agent in self.model.schedule.agents if isinstance(agent, LaborAgent) and not agent.employed]

        if len(unemployed_workers) > 0:
            # Hire one unemployed worker (randomly)
            worker = random.choice(unemployed_workers)
            worker.employed = True
            # Produce goods using the worker and other resources
            pass
        else:
            # No vacant job, the firm cannot produce more goods
            pass

        # Set the price for the goods
        self.price = random.uniform(10, 50)

class LaborAgent(Agent):
    _id_counter = 0  # Counter to assign unique IDs to labor agents

    def __init__(self, model, age):
        unique_id = LaborAgent._id_counter
        super().__init__(unique_id, model)
        self.age = age
        self.employed = False if age == 16 else True

        LaborAgent._id_counter += 1

    def step(self):
        # Update agent's age
        self.age += 1

        # Labor agents take action in each step (e.g., job search, getting fired)
        if self.age >= 65:
            # Retire the agent if they reach the retirement age
            self.model.schedule.remove(self)
        elif self.employed:
            # Agent has a job
            if random.random() < self.model.job_loss_probability:
                # Agent loses the job due to an exterior shock (e.g., economic crisis or technology advance)
                self.employed = False
        else:
            # Agent is unemployed and searches for a job
            if random.random() < self.model.job_search_probability:
                # Agent finds a new job
                self.employed = True

class EconomicModel(Model):
    def __init__(self, num_consumers, num_firms, num_labor_agents, job_search_probability, job_loss_probability, birth_rate, death_rate):
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(1, 1, False)
        self.datacollector = DataCollector(
            model_reporters={"UnemploymentRate": "get_unemployment_rate", "OldAgedDependencyRatio": "get_old_aged_dependency_ratio"}
        )
        self.job_search_probability = job_search_probability
        self.job_loss_probability = job_loss_probability
        self.birth_rate = birth_rate
        self.death_rate = death_rate

        # Create consumer agents (retirees)
        for i in range(num_consumers):
            age = random.randint(65, 100)
            propensity = random.uniform(0.1, 1.0)
            consumer = ConsumerAgent(i, self, age, propensity)
            self.schedule.add(consumer)
            self.grid.place_agent(consumer, (0, 0))

        # Create labor agents
        for i in range(num_labor_agents):
            age = random.randint(16, 64)
            propensity = random.uniform(0.1, 1.0)
            consumer = ConsumerAgent(i + num_consumers, self, age, propensity)
            self.schedule.add(consumer)
            self.grid.place_agent(consumer, (0, 0))

        # Create firm agents
        for i in range(num_firms):
            firm = FirmAgent(i, self, price=random.uniform(10, 50))
            self.schedule.add(firm)
            self.grid.place_agent(firm, (0, 0))

    def step(self):
        # Perform yearly population dynamics (birth and death)
        self.perform_population_dynamics()

        # Run agent step functions
        self.schedule.step()
        self.datacollector.collect(self)

    def perform_population_dynamics(self):
        # Perform birth and death rate calculations
        num_births = int(self.birth_rate * len([a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 16]))
        num_deaths = int(self.death_rate * len([a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 65]))
        
        # Create new labor agents (16-year-olds)
        for _ in range(num_births):
            new_agent = LaborAgent(self, age=16)
            self.schedule.add(new_agent)
            self.grid.place_agent(new_agent, (0, 0))
        
        # Remove retired agents (aged 65 and over)
        retire_agents = [a for a in self.schedule.agents if isinstance(a, ConsumerAgent) and a.age >= 65]
        for agent in retire_agents[:num_deaths]:
            self.schedule.remove(agent)

    def get_unemployment_rate(self):
        employed_agents = [agent for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.employed]
        total_labor_force = sum(1 for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.age >= 16 and agent.age <= 64)
        return 1 - len(employed_agents) / total_labor_force if total_labor_force > 0 else 0

    def get_old_aged_dependency_ratio(self):
        retirees = sum(1 for agent in self.schedule.agents if isinstance(agent, ConsumerAgent) and agent.age >= 65)
        labor_force = sum(1 for agent in self.schedule.agents if isinstance(agent, LaborAgent) and agent.age >= 16 and agent.age <= 64)
        return labor_force / retirees if retirees > 0 else 0

# Set the number of agents for each type
num_consumers = 50
num_firms = 10
num_labor_agents = 100
job_search_probability = 0.1  # Probability of finding a job when unemployed
job_loss_probability = 0.05   # Probability of losing a job when employed
birth_rate = 0.02            # Birth rate (percentage of new agents each year)
death_rate = 0.01            # Death rate (percentage of agents in the retired group that die each year)

# Initialize the model
model = EconomicModel(num_consumers, num_firms, num_labor_agents, job_search_probability, job_loss_probability, birth_rate, death_rate)

# Run the model for a certain number of steps (e.g., 50 years)
for _ in range(50):
    model.step()

# Access data collected during the simulation
data = model.datacollector.get_model_vars_dataframe()
print(data)


Exception: Agent with unique id 0 already added to scheduler